In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import date
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm.notebook import tqdm

from scripts import dl_utils
from scripts.dl_utils import predict_spectrogram, rect_from_point
from scripts.nn_predict import make_predictions, visualize_predictions

np.random.seed(1)

In [ ]:
train_data_dir = '../../data/training_data/fires/'
data_files = [f for f in os.listdir(train_data_dir) if '.pkl' in f and 'labels' not in f]

label_files = [f.split('.pkl')[0] + '_labels.pkl' for f in data_files]

patches = []
labels = []
for data, label in zip(data_files, label_files):
    with open(os.path.join(train_data_dir, data), 'rb') as f:
        data = pickle.load(f)
        for patch in data:
            patches.append(dl_utils.pad_patch(patch, 96))
    with open(os.path.join(train_data_dir, label), 'rb') as f:
        labels += pickle.load(f)
patches = np.array(patches)
labels = np.array(labels)

positive_patches = patches[labels == 1]
negative_patches = patches[labels == 0]

print("Loaded", len(positive_patches), "positive patches and", len(negative_patches), "negative patches")

In [ ]:
x = np.concatenate((positive_patches, negative_patches))
y = np.concatenate((np.ones(len(positive_patches)), np.zeros(len(negative_patches))))

x, y = shuffle(x, y, random_state=42)
x = np.array([dl_utils.unit_norm(patch) for patch in x])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
#x_train = np.expand_dims(x_train, -1)
#x_test = np.expand_dims(x_test, -1)

print("Num Train Samples:\t\t", len(x_train))
print("Num Test Samples:\t\t", len(x_test))
print(f"Percent Negative Train:\t {sum(y_train == 0.0) / len(y_train):.1%}")
print(f"Percent Negative Test:\t {sum(y_test == 0.0) / len(y_test):.1%}")
print(f"Input data shape: {x_train.shape}")

# Note: I am accustomed to assigning two classes for binary classification. 
# This habit comes from an issue in theano a long time ago, but I'm too superstitious to change it.
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
augmentation_parameters = {
    'featurewise_center': False,
    'rotation_range': 360,
    'width_shift_range': [0.9, 1.1],
    'height_shift_range': [0.9, 1.1],
    'shear_range': 10,
    'zoom_range': [0.8, 1.2],
    'vertical_flip': True,
    'horizontal_flip': True,
    # Fill options: "constant", "nearest", "reflect" or "wrap"
    'fill_mode': 'reflect'
}

datagen = ImageDataGenerator(**augmentation_parameters)


plt.figure(figsize=(12,12), facecolor=(1,1,1), dpi=150)
img, labels = datagen.flow(x_train, y_train, batch_size=64).next()
for index, (image, label) in enumerate(zip(img, labels)):
    rgb = (image[:,:,3:0:-1] + 1) / 4
    plt.subplot(8, 8, index+1)
    plt.imshow(np.clip(rgb, 0, 1))
    if label[1] == 1:
        plt.title('Fire')
    else:
        plt.title('No Fire')
    plt.axis('off')
plt.suptitle('Data Augmentation Examples')
plt.tight_layout()
plt.show()
    

In [ ]:
input_shape = np.shape(x_train[0])
model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(16, kernel_size=(3), padding='same', activation="relu"),
        layers.MaxPooling2D(pool_size=(2)),
        layers.Conv2D(32, kernel_size=(3), padding='same', activation="relu"),
        layers.MaxPooling2D(pool_size=(2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.6),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.6),
        layers.Dense(64, activation="relu"),
        #layers.Dense(32, activation="relu"),
        layers.Dropout(0.6),
        #layers.Dense(32, activation="relu"),
        layers.Dense(num_classes, activation="softmax")])
model.summary()

model.compile(loss="binary_crossentropy", 
              optimizer="adam", 
              metrics=["accuracy"])

train_accuracy = []
test_accuracy = []

In [ ]:
batch_size = 32
epochs = 128
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), 
        epochs=epochs, 
        validation_data = (x_test, y_test),
        verbose = 1
        )
train_accuracy += model.history.history['accuracy']
test_accuracy += model.history.history['val_accuracy']

In [ ]:
plt.figure(figsize=(8,5), dpi=100, facecolor=(1,1,1))
plt.plot(train_accuracy, label='Train Acc')
plt.plot(test_accuracy, c='r', label='Val Acc')
percent_negative = (sum(y_train == 0.0) / len(y_train))[1]
plt.plot([0, epochs-1], [percent_negative, percent_negative], '--', c='gray', label='Baseline')
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Network Train and Val Accuracy')
plt.show()

# Test Model

In [ ]:
preds = model.predict(x_test)[:,1]
threshold = 0.5
binary_preds = (preds > threshold).astype(int)
failure_index = np.where(binary_preds != y_test[:,1].astype(int))[0]
print(len(failure_index), "failures")

In [ ]:
for i in failure_index:
    plt.imshow(np.clip((x_test[i] + 1) / 3, 0, 1)[:,:,3:0:-1])
    plt.title(f"Pred: {preds[i]:.2f}. Label {y_test[i,1]}")
    plt.axis('off')
    plt.show()